<a href="https://colab.research.google.com/github/SebQuispe/Trabajo_API_LP2/blob/rama_Alexander/escrapeo_curacao_actualizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os

# Cargar el archivo HTML
with open("celulares_curacao.html", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# Lista para guardar resultados
productos = []

# Función para limpiar precios
def limpiar_precio(texto):
    if texto:
        texto = texto.replace("S/", "").replace("\xa0", "").replace(",", "").strip()
        try:
            return float(texto)
        except:
            return None
    return None

# Buscar productos
for item in soup.select("li.product-item"):
    nombre = item.select_one("a.product-item-link")
    precio_actual = item.select_one("span.price")
    precio_anterior = item.select_one("span.old-price .price")

    productos.append({
        "modelo": nombre.get_text(strip=True) if nombre else None,
        "precio_actual": limpiar_precio(precio_actual.get_text(strip=True)) if precio_actual else None,
        "precio_anterior": limpiar_precio(precio_anterior.get_text(strip=True)) if precio_anterior else None
    })

# Convertir a DataFrame
df = pd.DataFrame(productos)

# Cálculo de descuentos
df["diferencia"] = df["precio_anterior"] - df["precio_actual"]
df["porcentaje_descuento"] = round((df["diferencia"] / df["precio_anterior"]) * 100, 2)

# Mostrar primeras filas
print(df.head())

# Crear carpeta /data si no existe
os.makedirs("data", exist_ok=True)

# Guardar CSV
df.to_csv("data/productos_curacao.csv", index=False, encoding="utf-8-sig")
print("\n✅ Archivo 'productos_curacao.csv' guardado con columnas de descuento en la carpeta /data")


                                              modelo  precio_actual  \
0         Celular Samsung Galaxy A56 256GB 12GB 6.7"         1649.0   
1  Celular Samsung Galaxy S25 Ultra 256GB 12GB 6....         3999.0   
2  Apple iPhone 16 Pro Max 256GB 8GB 6.9" Natural...         4999.0   
3  Celular Honor Magic 7 Lite 256GB 8GB 6.7" Tita...         1319.0   
4  Celular Xiaomi Poco X7 Pro 512GB 12GB 6.67" Negro         1449.0   

   precio_anterior  diferencia  porcentaje_descuento  
0           2059.0       410.0                 19.91  
1           5269.0      1270.0                 24.10  
2           6159.0      1160.0                 18.83  
3           1749.0       430.0                 24.59  
4           1999.0       550.0                 27.51  

✅ Archivo 'productos_curacao.csv' guardado con columnas de descuento en la carpeta /data
